# YouTube to MP3 Converter (Colab Ready)
This notebook allows you to paste a YouTube link, select a desired length for the clip, and convert it to a downloadable MP3 file with basic metadata tagging.

In [ ]:
# Install dependencies
!pip install pytube pydub mutagen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 8.1 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube
from pydub import AudioSegment
from mutagen.easyid3 import EasyID3
from mutagen.mp3 import MP3
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
import os

In [ ]:
# UI widgets
length_dropdown = widgets.Dropdown(
    options=[30, 60, 90, 120, 180, 300],
    value=60,
    description='MP3 Length:',
    style={'description_width': 'initial'}
)

url_input = widgets.Text(
    value='',
    placeholder='Enter YouTube URL here',
    description='YouTube URL:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

download_button = widgets.Button(
    description='Convert and Download MP3',
    button_style='success'
)

display(length_dropdown, url_input, download_button)

Dropdown(description='MP3 Length:', index=1, options=(30, 60, 90, 120, 180, 300), style=DescriptionStyle(descr…

Text(value='', description='YouTube URL:', layout=Layout(width='70%'), placeholder='Enter YouTube URL here', s…

Button(button_style='success', description='Convert and Download MP3', style=ButtonStyle())

In [ ]:
def on_button_click(b):
    url = url_input.value
    clip_length = length_dropdown.value

    if not url:
        print("Please enter a valid YouTube URL.")
        return

    print("Downloading audio stream...")  # Indicate download start
    yt = YouTube(url)
    stream = yt.streams.filter(only_audio=True).first()
    out_file = stream.download(filename='temp_audio')

    print("Trimming and exporting MP3...")  # Indicate trimming start
    audio = AudioSegment.from_file(out_file)
    trimmed = audio[:clip_length * 1000]
    output_filename = f"{yt.title[:50].strip().replace(' ', '_')}.mp3"
    trimmed.export(output_filename, format="mp3")
    os.remove(out_file)

    print("Tagging metadata...")  # Indicate tagging start
    try:
        audiofile = MP3(output_filename, ID3=EasyID3)
        audiofile['title'] = yt.title
        audiofile['artist'] = yt.author
        audiofile['album'] = 'YouTube'
        audiofile.save()
    except Exception as e:
        print(f"Metadata tagging failed: {e}")

    print(f"Download ready: {output_filename}")
    files.download(output_filename)